In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import utulek

utulek.platform.import_globals_notebook(globals())
! ls {utulek.ASSETS_PATH}

base_module = utulek


/home/gilgamesh/main.syncthing/utulek/.venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


IS_NOTEBOOK_KERNEL_CODE = True
NOTEBOOK_NAME = 2026_01_01-whisper.3.ipynb
ASSETS_PATH = /home/gilgamesh/main.syncthing/utulek/experiment/2026_01_01-whisper.ipynb.assets/
Loaded `/home/gilgamesh/main.syncthing/utulek/experiment/.env`.
torch: ['NVIDIA GeForce GTX 1080 Ti']
tf: /device:GPU:0
jax: [CudaDevice(id=0)]
Monologue_The_Year_Ahead.mp3
podcast.1.opus
podcast.opus
The_Anti-Consumer_Electronics_Show_with_Steve_Burke.mp3
The_Enshittifinancial_Crisis_Part_Four.mp3
The_Enshittifinancial_Crisis_Part_One.mp3
The_Enshittifinancial_Crisis_Part_Three.mp3
The_Enshittifinancial_Crisis_Part_Two.mp3
Working_In_The_Dot_Com_Bubble_ft_Matt_Rosoff.mp3


I0000 00:00:1769475797.033505 2201642 gpu_device.cc:2020] Created device /device:GPU:0 with 9339 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1
/home/gilgamesh/.pyenv/versions/3.11.9/lib/python3.11/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [3]:
def score_audio_segments(
    classifier,
    audio: np.ndarray,
    sr: int,
    candidate_labels: typing.List[str],
    context_len_ms: int = 16000,
    segment_overlap_ms: int = 4000,
):
    audio_padded = np.hstack(
        [audio, np.zeros((segment_overlap_ms - len(audio) % segment_overlap_ms,))]
    )
    segments = [
        audio_padded[i : i + context_len_ms * sr // 1000]
        for i in range(
            0,
            len(audio_padded) - (context_len_ms - segment_overlap_ms) * sr // 1000,
            segment_overlap_ms * sr // 1000,
        )
    ]
    scores = {i: np.zeros((len(segments),)) for i in candidate_labels}
    for i in tqdm(range(len(segments))):
        result = classifier(segments[i], candidate_labels=candidate_labels)
        for j in result:
            scores[j["label"]][i] = j["score"]

    return scores

In [4]:
def compute_p_keep_threshold(p_keep: np.ndarray):
    # We can compute threshold directly via CDF.
    p_keep_sorted = sorted(p_keep)

    # Take the lowest slope in a reasonable range.
    SLOPE_X_EPS = 0.05
    slopes = [None] * len(p_keep)
    for i in range(len(p_keep)):
        est_begin = est_end = i
        while (
            est_begin >= 0
            and p_keep_sorted[est_begin] + SLOPE_X_EPS / 2 >= p_keep_sorted[i]
        ):
            est_begin -= 1
        while (
            est_end < len(p_keep)
            and p_keep_sorted[est_end] - SLOPE_X_EPS / 2 <= p_keep_sorted[i]
        ):
            est_end += 1
        est_begin += 1
        est_end -= 1
        # Add a small EPS in case of divide by 0.
        slopes[i] = (est_end - est_begin) / (
            p_keep_sorted[est_end] - p_keep_sorted[est_begin] + 1e-9
        )

    SLOPE_IGNORE_BOUNDARY_PERCENTILE = 0.1
    threshold_i = np.argmin(
        slopes[
            int(len(slopes) * SLOPE_IGNORE_BOUNDARY_PERCENTILE) : -int(
                len(slopes) * SLOPE_IGNORE_BOUNDARY_PERCENTILE
            )
        ]
    )
    threshold = p_keep_sorted[
        int(len(slopes) * SLOPE_IGNORE_BOUNDARY_PERCENTILE) + threshold_i
    ]

    return threshold

In [5]:
def compute_cuts_keeps(
    p_keep: np.ndarray, threshold: float, context_range_segs: int = 8
):
    median_keep = (
        np.asarray(
            [
                np.percentile(
                    p_keep[max(0, i - context_range_segs) : i + context_range_segs], 25
                )
                for i in range(len(p_keep))
            ]
        )
        > threshold
    )

    cut_segs = np.hstack([-1, np.where(median_keep == 0)[0], len(median_keep)])
    keep_segs = np.hstack([-1, np.where(median_keep != 0)[0], len(median_keep)])
    keep_breaks = np.where(keep_segs + 1 != np.roll(keep_segs, -1))[0][:-1]
    cuts = keep_segs[keep_breaks] + 1, keep_segs[keep_breaks + 1]
    cut_breaks = np.where(cut_segs + 1 != np.roll(cut_segs, -1))[0][:-1]
    keeps = cut_segs[cut_breaks] + 1, cut_segs[cut_breaks + 1]
    return (cuts, keeps)

In [6]:
def cut_ads(classifier, audio_path):
    CANDIDATE_LABELS = ["advertisement", "podcast"]
    CONTEXT_LEN_MS = 16000
    SEGMENT_OVERLAP_MS = 4000
    CONTEXT_RANGE_SEGS = 12

    audio, sr = librosa.load(audio_path, sr=None)

    scores = score_audio_segments(
        classifier,
        audio,
        sr,
        candidate_labels=CANDIDATE_LABELS,
        context_len_ms=CONTEXT_LEN_MS,
        segment_overlap_ms=SEGMENT_OVERLAP_MS,
    )
    p_keep = scores["podcast"]
    threshold = compute_p_keep_threshold(p_keep)
    cuts, keeps = compute_cuts_keeps(
        p_keep, threshold, context_range_segs=CONTEXT_RANGE_SEGS
    )

    audio_keep = np.hstack(
        [
            audio[
                keeps[0][i]
                * SEGMENT_OVERLAP_MS
                * sr
                // 1000 : keeps[1][i]
                * SEGMENT_OVERLAP_MS
                * sr
                // 1000
            ]
            for i in range(len(keeps[0]))
        ]
    )

    basename = os.path.basename(audio_path).rsplit(".", 1)[0]
    soundfile.write(f".data/{basename}.wav", audio_keep, sr)
    print(f"{basename}: kept {int(len(audio_keep) / len(audio) * 100)}%.")

In [11]:
classifier = transformers.pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)

Device set to use cuda:0


In [12]:
for i in glob.glob(utulek.ASSETS_PATH + "*.mp3"):
    cut_ads(classifier, i)

[src/libmpg123/id3.c:process_comment():587] error: No comment text / valid description?


  0%|          | 0/994 [00:00<?, ?it/s]

Working_In_The_Dot_Com_Bubble_ft_Matt_Rosoff: kept 69%.


[src/libmpg123/id3.c:process_comment():587] error: No comment text / valid description?


  0%|          | 0/1054 [00:00<?, ?it/s]

The_Anti-Consumer_Electronics_Show_with_Steve_Burke: kept 84%.
